##Paquetes

In [ ]:
#Importar los paquetes necesarios para manejar la información y utilizar la API de NPClassifier
import pandas as pd
import requests

##Carga del CSV

In [ ]:
df = pd.read_csv('file.csv', sep = ',') #Subir el archivo al entorno y cambiar el nombre y el separador según sea necesario
df.head() #Revisar las primeras filas para comprobar que se cargó correctamente

##NPClassifier

In [ ]:
global failed_smiles
failed_smiles = {} #Diccionario para almacenar los SMILES que no den respuesta por parte de la API

def NPC_API_call(SMILES, rownum): #Solicitud a la API

  url = 'https://npclassifier.gnps2.org/classify?smiles=' + SMILES  #URL para llamar a la API de NPClassifier

  try:
      NPC_API_response = requests.get(url)
      NPC_API_response.raise_for_status() #Revisa si la solicitud resultó en un error
      return NPC_API_response.json()

  except requests.exceptions.RequestException as e: #Muestra un mensaje de error si no hay respuesta
      print(f"Request error: row {rownum}, {SMILES}... {e}") #Indica la fila, el SMILES y el tipo de error
      failed_smiles[rownum] = SMILES
      return None

In [ ]:
def NPC_data_parse(df): #Toma los SMILES del df y las procesa con la función definida antes
  failed_smiles.clear() #Limpia el diccionario de errores
  for index, row in df.iterrows():
      tossed_smiles = row['SMILES']  #Toma el SMILES de cada fila
      NPC_data = NPC_API_call(tossed_smiles, index) #Lo usa como argumento para la función de la solicitud a la API

      if NPC_data:
          for i, result in enumerate(NPC_data.get('pathway_results')): #Guarda los resultados de rutas metabólicas
              df.at[index, f'NPC_PATHWAY_{i+1}'] = result

          for i, result in enumerate(NPC_data.get('superclass_results')): #Guarda los resultados de superclases
              df.at[index, f'NPC_SUPERCLASS_{i+1}'] = result

          for i, result in enumerate(NPC_data.get('class_results')): #Guarda los resultados de clases
              df.at[index, f'NPC_CLASS_{i+1}'] = result

          df.at[index, 'NPC_IS_GLYCOSIDE'] = NPC_data.get('isglycoside') #Guarda el booleano sobre si el compuesto es un glucósido

In [ ]:
#Corre la función por el df
NPC_data_parse(df)

In [ ]:
def reorder_columns(df): #Para acomodar las columnas en el orden correcto. Facilita la entrada manual
    pathway_cols = [col for col in df.columns if col.startswith('NPC_PATHWAY_')] #Guarda los nombres de columnas que corresponden a rutas
    superclass_cols = [col for col in df.columns if col.startswith('NPC_SUPERCLASS_')] #Guarda los nombres de columnas que corresponden a superclases
    class_cols = [col for col in df.columns if col.startswith('NPC_CLASS_')] #Guarda los nombres de columnas que corresponden a clases

    #Obtienen el número asignado a cada columna y las ordena según este
    pathway_cols_sorted = sorted(pathway_cols, key=lambda x: int(x.split('_')[-1]))
    superclass_cols_sorted = sorted(superclass_cols, key=lambda x: int(x.split('_')[-1]))
    class_cols_sorted = sorted(class_cols, key=lambda x: int(x.split('_')[-1]))

    other_cols = [col for col in df.columns if col not in pathway_cols and col not in superclass_cols and col not in class_cols] #Todas las demás columnas

    new_column_order = other_cols + pathway_cols_sorted + superclass_cols_sorted + class_cols_sorted #Coloca primero todas las otras columnas, luego las de rutas, luego superclases y por último clases

    return df[new_column_order]

df = reorder_columns(df)

In [ ]:
def NPC_manual_entry(): #Para los SMILES que no dan respuesta
  classification_results = [] #Crea una lista para guardar las columnas con información de NPClassifier. Dependiendo del set puede tener diferentes cantidades de columnas de rutas, superclases y clases
  for col in df.columns:
    if col.startswith('NPC'):
      classification_results.append(col)
  classification_results.remove('NPC_IS_GLYCOSIDE') #Elimina la columna de si es glucósido

  for i in failed_smiles.keys():
    print(f'Row {i}\nSMILES:\n{failed_smiles[i]}') #Imprime la fila y el SMILES para copiarlo y pegarlo en NPClassifier
    for colname in classification_results:
      classification_inputs = input(f'{colname} (Enter to leave as None): ').strip() or None #Pide ingresar manualmente la información
      df.at[i, colname] = classification_inputs #La guarda en la columna respectiva

    glycoside_check = input("Is glycoside? (1 = True, 0 = False): ").strip() #Pide la información sobre glicósidos
    if glycoside_check == '1':
      glycoside_check = True
    elif glycoside_check == '0':
      glycoside_check = False
    else:
      raise ValueError("Invalid input. Please enter either '1' or '0'.") #Revisa que el valor digitado sea un '0' (no) o un '1' (sí)
    df.at[i, 'NPC_IS_GLYCOSIDE'] = glycoside_check
    print(df.iloc[i]) #Muestra la información actualizada

In [ ]:
NPC_manual_entry() #Corre la función de entrada manual